### Sentiment Analysis with CNN

##### Introduction and Preprocessing

本节参考 [斗大的熊猫](http://blog.topspeedsnail.com/archives/10420) ，使用 CNN 在 [Sentiment140](http://help.sentiment140.com) 数据集上进行情感分析

Sentiment140 数据集包含 1,600,000 个训练 tweets 数据和 498 个测试数据，标签包含消极、中性和积极tweet

数据格式：移除表情符号的CSV文件，字段如下：
```
0 – the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)，即标签 Y
1 – the id of the tweet (2087)，无用
2 – the date of the tweet (Sat May 16 23:58:44 UTC 2009)，无用
3 – the query (lyx). If there is no query, then this value is NO_QUERY，无用
4 – the user that tweeted (robotickilldozr)，无用
5 – the text of the tweet (Lyx is cool)，文本，即训练数据 X
```

由于文件很大，无用字段很多，故此使用预处理脚本删除掉无用字段，同时还会生成全词汇数组文件 lexicon.pickle

> python2.7 sentiment140_preprocess.py

运行后得到 3 个结果文件

- data/sentiment140/lexicon.pickle  是由得到的 7176 个 lexicon 词典数组经过 cPickle.dump 得到的文件
- data/sentiment140/traindata   格式如 [1, 0, 0]:%:%:%[5217,2341,13]:%:%:%how are you，表示 positive，3个词在词典中位置为 5217,2341,13，不过顺序无关，也就是说 how 不一定在 5217 位置
- data/sentiment140/testdata   格式同上

上面的脚本需要使用 nltk 模块，且安装 nltk.download('punkt') 和 nltk.download('wordnet')

最后的 lexicon file 中共计 Total word count in lexicon: 7176 个词

然后还实现了一个 python module，叫做 sentiment140.py ，用于处理训练和测试数据，生成喂给 CNN 的训练和测试数据，主要实现两个方法

- get_train_batch(n=150)
- get_test_dataset()

这两个方法都返回 X, Y 两个数组

- X 数组中的每个元素也是一个数组，对应一条 tweet，长度为词典 lex 总数，该 tweet 在词典中存在的词的索引为 1， 否则为 0，即词 one-hot 编码之和
- Y 数组中的每个元素也是一个数组，对应一条 tweet 的标签，[1, 0, 0] 为 positive [0, 1, 0] 为 neutral [0, 0, 1] 为 negative

In [2]:
import tensorflow as tf
import numpy as np
import cPickle

from sentiment140 import Sentiment140

In [3]:
tf.__version__

'0.9.0'

##### Part I. 探索数据集

In [4]:
senti = Sentiment140()
test_x, test_y = senti.get_test_dataset()
print len(test_x), len(test_x[0])
print len(test_y), len(test_y[0])

498 7176
498 3


看到测试集中 498 个记录；每条记录的 x 为 7176 维，也即词典长度，y 为 3 维，也即 positive/neutral/negative

训练集过大，故此没有一次性导入，Sentiment140.py 提供一个取随机 batch 的函数；维度和上面是一致的

In [5]:
input_size = len(senti.lex)    # 这里就不去重新加载 lex 词典了，因为都已经在 Sentiment140 类中封装好了，故此通过这个方法取词典长度
num_classes = 3
print input_size

7176



##### Part II. Layer definations

In [6]:
# 定义参数
filter_sizes = [3, 4, 5]   # 3 层 CNN 对应 3 个 filters，每个 filters 都是一维，或者理解为 n x 1 的二维
num_filters = 128          # 每层 filter 都是 128 组，提取不同的特征
batch_size = 90    # 测试集 batch size

后面要做 embedding，故此这里先熟悉一下 tf.nn.embedding_lookup 函数
```
>>> sess = tf.InteractiveSession()
>>> w = tf.random_uniform([5, 2], -1.0, 1.0)       # 这里假设词典中词个数为 5 维，embedding 到 2 维，初始化为 -1~1 之间的随机数
>>> w = sess.run(w)
>>> x = [[1,1,0,0,0], [1,0,0,0,0], [0,0,1,1,1]]     # x 为 3 x 5 维， 3 代表 3 个样本，5 为词典中词个数，1 表示该词在词典中，0 表示不在
>>> embedded_chars = tf.nn.embedding_lookup(w, x)

>>> w
array([[ 0.82283998,  0.21245265],              # 看到，w 确实是随机出来了，每一行表示词典中的对应词的 2 维 embedding 向量
       [-0.737818  , -0.59785843],
       [-0.24692678, -0.69566345],
       [ 0.85945463, -0.21308041],
       [ 0.28053808,  0.88169646]], dtype=float32)
>>> x
[[1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 1, 1]]
>>> sess.run(embedded_chars)
array([[[-0.737818  , -0.59785843],             # 注意， embedding_chars 为 3 x 5 x 2 维，即样本个数 x 词典中词总个数 x embedding 维度
        [-0.737818  , -0.59785843],
        [ 0.82283998,  0.21245265],           # 我们以 embedding_chars[0::] 为例，也就是第一个样本得到的 lookup 结果
        [ 0.82283998,  0.21245265],           # 第一个样本为 [1,1,0,0,0]，即该样本中有词典的前 2 个词，而没有后面 3 个词
        [ 0.82283998,  0.21245265]],           # 我原以为 embedding_chars[0::] 会是前两个词的 embedding 向量 + 3 个 [0,0] 
                                    # 结果并不是这样，1 的部分填入 w[1]，而 0 的部分填入 w[0]
       [[-0.737818  , -0.59785843],           # 这样有一个问题，那就是 w[2:] 也就是 w 矩阵从第三个开始往后的元素都没有用到啊！
        [ 0.82283998,  0.21245265],           
        [ 0.82283998,  0.21245265],           # 保留疑问，后面再说
        [ 0.82283998,  0.21245265],
        [ 0.82283998,  0.21245265]],

       [[ 0.82283998,  0.21245265],
        [ 0.82283998,  0.21245265],
        [-0.737818  , -0.59785843],
        [-0.737818  , -0.59785843],
        [-0.737818  , -0.59785843]]], dtype=float32)
```

In [7]:
def neural_network(data, dropout_keep_prob):
    """
    data 即 batch of X，维度为 number_of_data x input_size
    """
    # embedding 层，把 one-hot 累计和的输入转为 vector
    with tf.name_scope("embedding"):
        embedding_size = 128
        # W 在 -1 & 1 之间随机分布，维度为 input_size x embedding_size
        W = tf.Variable(tf.random_uniform([input_size, embedding_size], -1.0, 1.0))
        # 见上一个 cell 中的研究，有疑问，先继续再说，embedded_char 为 sample_count x input_size x embedding_size 维
        embedded_chars = tf.nn.embedding_lookup(W, data)
        # 在最后再添加一个维度，那么就是 sample_count x input_size x embedding_size x 1 维
        # 我们知道 cnn 的输入为 sample_count x image_width x image_height x num_channel ，这里 num_channel 为 1
        # 然后，input_size x embedding_size 就相当于图片中的 width x height
        embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

    # 接下来是 CNN 层，注意和之前图片的串行 CNN 不同，这里采用的是并行 CNN，就是从 embedding 层出来的结果同时进入 3 个 CNN
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-{}-{}".format(i, filter_size)):
            # 准备初始化权重，首先得到权重的 shape
            # 输入的每个样本为 input_size x embedding_size ，那么我们把每层 filter 的维度设计为 filter_size x embedding_size
            # 也就是说，filter 的一个维度和样本相当，故此 filter 只会在 input_size 这个维度上滑动
            # 每层的 filter 都有 num_filters 组，而输入的 num_channel 都为 1
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]))
            # 每次移动一步
            conv = tf.nn.conv2d(embedded_chars_expanded, W, strides=[1, 1, 1, 1], padding="VALID")
            h = tf.nn.relu(tf.nn.bias_add(conv, b))
            # pooling，pooling 跨度很大，input_size - filter_size + 1, 也就是说 input_size 那么长的向量，pool 完之后只剩 filter_size 长
            pooled = tf.nn.max_pool(h, ksize=[1, input_size - filter_size + 1, 1, 1], strides=[1,1,1,1], padding="VALID")
            pooled_outputs.append(pooled)
    
    # 前面说道，每层 CNN 对每个样本都会得到 num_filters 组结果；那么把各个样本的结果 concate 起来
    num_filters_total = num_filters * len(filter_sizes)
    h_pool = tf.concat(3, pooled_outputs)
    # 看到结果为若干个 num_filters_total 矢量，每个矢量都是不同样本的结果 concat 的结果
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
    
    # dropout
    with tf.name_scope("dropout"):
        h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)
        
    # output full connection 层
    with tf.name_scope("output"):
        # W = tf.get_variable("W", shape=[num_filters_total, num_classes], initializer=tf.contrib.layers.xavier_initializer())
        W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.5))
        b = tf.Variable(tf.constant(0.1, shape=[num_classes]))
        output = tf.nn.xw_plus_b(h_drop, W, b)
    return output

##### Part III. Trainning

In [8]:
X = tf.placeholder(tf.int32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, num_classes])
dropout_keep_prob = tf.placeholder(tf.float32)

In [19]:
def train():
    output = neural_network(X, dropout_keep_prob)
    optimizer = tf.train.AdamOptimizer(1e-3)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, Y))
    optimizer = optimizer.minimize(loss)
    
    y_pred = tf.argmax(output, 1)
    correct_predictions = tf.equal(y_pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        i = 0
        pre_accuracy = 0
        no_improvement = 0
        while True:
            i += 1
            print "-------   iter {}  ------".format(i)
            batch_x, batch_y = senti.get_train_batch(n=batch_size)
            sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y, dropout_keep_prob:0.5})
            
            if i % 10 == 0:
                ccur = sess.run(accuracy, feed_dict={X:test_x[0:50], Y:test_y[0:50], dropout_keep_prob:1.0})
                print('accuracy:', accur)
                if accur > pre_accuracy:
                    no_improvement = 0
                    pre_accuracy = accur
                    saver.save(sess, 'data/sentiment140/model.ckpt')
                else:
                    no_improvement += 1
                    if no_improvement == 5:
                        break


In [ ]:
train()

-------   iter 1  ------
-------   iter 2  ------
